In [1]:

## rootfinder (basic and fast versions)
import torch
torch.set_default_dtype(torch.float64)
from torchspace.rootfinder.methods import newton, bisect
from torchspace.rootfinder.roots import get_u_parameter, get_xi_parameter
import time

# Rambo mass function test

In [2]:
n = int(1e6)
particles = 5
ptemp = 1000*torch.randn((n,particles-1,4))
plast = -ptemp.sum(dim=1, keepdim=True)
p = torch.cat([ptemp, plast], dim=1)
m = 500*torch.rand((1,particles))
pmag2 = torch.sum(p[:,:,1:]**2, dim=-1)
p[:,:,0] = torch.sqrt(pmag2 + m**2)
p_tot =  torch.sum(p, dim=1)
e_cm = p_tot[:,0]

# define the function and its gradient with respect to x and p_i
def func2(x, p, m):
    root = torch.sqrt(x[:, None] ** 2 * p[:, :, 0] ** 2 + m**2)
    f = torch.sum(root, dim=-1) - e_cm
    return f
    
def dfunc2(x, p, m):
    root = torch.sqrt(x[:, None] ** 2 * p[:, :, 0] ** 2 + m**2)
    return torch.sum(x[:, None] * p[:, :, 0] ** 2 / root, dim=-1)

def dpfunc2(x, p, m):
    root = torch.sqrt(x[:, None] ** 2 * p[:, :, 0] ** 2 + m**2)
    return x[:, None]**2 * p[:, :, 0] / root

# set up the parameters and the initial guess
xi0 = 0.5*torch.ones((n,)).requires_grad_() # zeros as the initial guess

## Test method itself (newton)

In [3]:
q = p.clone().requires_grad_()
f = lambda x: func2(x, q,m)
df = lambda x: dfunc2(x,q,m)

min = (0.0) * torch.ones_like(xi0)
max = (1.0) * torch.ones_like(xi0)
start_time = time.time()
xi2 = newton(f,df,min,max,xi0)
print("--- %s seconds ---\n" % (time.time() - start_time))
Lxi = xi2.sum()

--- 0.4383690357208252 seconds ---



In [4]:
q.grad = None
start_time = time.time()
Lxi.backward(retain_graph=True)
print("--- %s seconds ---\n" % (time.time() - start_time))
qgrad1 = q.grad
print(qgrad1[5123,:,0])
print(qgrad1.shape)

--- 1.1457457542419434 seconds ---

tensor([-8.3012e-05, -8.3176e-05, -8.2560e-05, -8.2462e-05, -8.2438e-05])
torch.Size([1000000, 5, 4])


## Test autograd solver function

In [5]:

# new root finder
q = p.clone().requires_grad_()
start_time = time.time()
xi3 = get_xi_parameter(q[:,:,0],m)
print("--- %s seconds ---\n" % (time.time() - start_time))

--- 0.29462504386901855 seconds ---



In [6]:
q.grad = None
L = xi3.sum()
start_time = time.time()
L.backward(retain_graph=True)
print("--- %s seconds ---\n" % (time.time() - start_time))
qgrad2 = q.grad
print(qgrad2[5123,:,0])
print(qgrad2.shape)

--- 0.06886100769042969 seconds ---

tensor([5.8496e-07, 4.2081e-07, 1.0368e-06, 1.1352e-06, 1.1591e-06])
torch.Size([1000000, 5, 4])


# Rambo normal function

In [7]:
nparticles=5
n = int(1e5)

i = torch.arange(2, nparticles)[None, :]
xs = torch.rand((n, 3*nparticles-4), requires_grad=True)
def func3(x, xs):
    f=(
        (nparticles + 1 - i) * x ** (2 * (nparticles - i))
        - (nparticles - i) * x ** (2 * (nparticles + 1 - i))
        - xs[:, : nparticles - 2]
    )
    return f

def func3a(x, xs):
    f=(
        (nparticles + 1 - i) * x ** (2 * (nparticles - i))
        - (nparticles - i) * x ** (2 * (nparticles + 1 - i))
        - xs
    )
    return f

def dfunc3(x):
    df = (nparticles + 1 - i) * (2 * (nparticles - i)) * x ** (
            2 * (nparticles - i) - 1
        ) - (nparticles - i) * (2 * (nparticles + 1 - i)) * x ** (
            2 * (nparticles + 1 - i) - 1
        )
    return df

# set up the parameters and the initial guess
u0 = 0.5*torch.ones((n, nparticles-2)) # zeros as the initial guess

In [8]:
xs.grad = None
f = lambda x: func3(x, xs)
df = lambda x: dfunc3(x)

min = (0.0) * torch.ones_like(u0)
max = (1.0) * torch.ones_like(u0)
start_time = time.time()
u2 = newton(f,df,min,max,u0)
print("--- %s seconds ---\n" % (time.time() - start_time))

--- 0.14864611625671387 seconds ---



In [9]:
# Test speed of backpropagation
L = u2.sum()
start_time = time.time()
L.backward()
print("--- %s seconds ---\n" % (time.time() - start_time))

--- 0.12767982482910156 seconds ---



## Autograd Solver

In [10]:
# Combine newton with custom Autograd 
xs.grad = None
start_time = time.time()
u4 = get_u_parameter(xs[:, :nparticles - 2])
print("--- %s seconds ---\n" % (time.time() - start_time))

--- 0.13277912139892578 seconds ---



In [11]:
L = u4.sum()
start_time = time.time()
L.backward()
print("--- %s seconds ---\n" % (time.time() - start_time))

--- 0.0008289813995361328 seconds ---



## Bisection Solver

In [12]:
# Test bisection
min = (0.0) * torch.ones_like(u0)
max = (1.0) * torch.ones_like(u0)
start_time = time.time()
u3 = bisect(f,df,min,max,u0)
print("--- %s seconds ---\n" % (time.time() - start_time))

--- 0.1539597511291504 seconds ---

